# Prep

In [ ]:
import pandas as pd

file_path = '../preped.csv'
df = pd.read_csv(file_path)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor

# Select features and target
features = ['Hidden Gem Score', 'Runtime', 'Awards Received', 'Awards Nominated For',
           'Boxoffice', 'IMDb Votes', 'Minimum Age'] + \
           [col for col in df.columns if col in ['Action', 'Adventure', 'Animation', 
           'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 
           'Fantasy', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 
           'News', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']]

target = 'IMDb Score'

# Preprocessing
X = df[features]
y = df[target]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train Models

In [ ]:
mlp = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)

# Make predictions
y_pred_imdb = mlp.predict(X_test)

# Evaluate model
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred_imdb))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred_imdb))
print(f'R2 Score: {r2_score(y_test, y_pred_imdb):.3f}')

In [ ]:
y_rt = df['Rotten Tomatoes Score']

# Split data (using same X from before)
X_train_rt, X_test_rt, y_train_rt, y_test_rt = train_test_split(X, y_rt, test_size=0.2, random_state=42)

# Create and train MLP model
mlp_rt = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
mlp_rt.fit(X_train, y_train_rt)

# Make predictions
y_pred_rt = mlp_rt.predict(X_test)

# Print metrics for Rotten Tomatoes
print('\nMetrics for Rotten Tomatoes Score:')
print('Mean Absolute Error:', f'{mean_absolute_error(y_test_rt, y_pred_rt):.3f}')
print('Mean Squared Error:', f'{mean_squared_error(y_test_rt, y_pred_rt):.3f}')
print('R2 Score:', f'{r2_score(y_test_rt, y_pred_rt):.3f}')

In [ ]:
y_mc = df['Metacritic Score']

# Split data (using same X from before)
X_train_mc, X_test_mc, y_train_mc, y_test_mc = train_test_split(X, y_mc, test_size=0.2, random_state=42)

# Create and train MLP model
mlp_mc = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
mlp_mc.fit(X_train, y_train_mc)

# Make predictions
y_pred_mc = mlp_mc.predict(X_test)

# Print metrics for Metacritic
print('\nMetrics for Metacritic Score:')
print('Mean Absolute Error:', f'{mean_absolute_error(y_test_mc, y_pred_mc):.3f}')
print('Mean Squared Error:', f'{mean_squared_error(y_test_mc, y_pred_mc):.3f}')
print('R2 Score:', f'{r2_score(y_test_mc, y_pred_mc):.3f}')

# Visualize Models

In [ ]:
import matplotlib.pyplot as plt

# Create a figure with subplots
plt.figure(figsize=(15, 5))

# Plot IMDb scores
plt.subplot(131)
plt.scatter(y_test, y_pred_imdb, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual IMDb Score')
plt.ylabel('Predicted IMDb Score')
plt.title('IMDb Scores')

# Plot Metacritic scores 
plt.subplot(132)
plt.scatter(y_test_mc, y_pred_mc, alpha=0.6)
plt.plot([y_test_mc.min(), y_test_mc.max()], [y_test_mc.min(), y_test_mc.max()], 'r--', lw=2)
plt.xlabel('Actual Metacritic Score')
plt.ylabel('Predicted Metacritic Score')
plt.title('Metacritic Scores')

# Plot Rotten Tomatoes scores
plt.subplot(133)
plt.scatter(y_test_rt, y_pred_rt, alpha=0.6)
plt.plot([y_test_rt.min(), y_test_rt.max()], [y_test_rt.min(), y_test_rt.max()], 'r--', lw=2)
plt.xlabel('Actual Rotten Tomatoes Score') 
plt.ylabel('Predicted Rotten Tomatoes Score')
plt.title('Rotten Tomatoes Scores')

plt.tight_layout()
plt.show()

# Optimize using HalvingGridSearchCV

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

# Define the parameter grid for HalvingGridSearchCV
param_grid = {
    'hidden_layer_sizes': [(50,), (100, 50), (200, 100)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive']
}

# Define the target columns
target_im = 'IMDb Score'
target_rt = 'Rotten Tomatoes Score'
target_mc = 'Metacritic Score'

targets = {
    target_im: {},
    target_rt: {},
    target_mc: {}
}

for target_name, target_data in targets.items():
    y = df[target_name]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    targets[target_name]['X_train_scaled'] = X_train_scaled
    targets[target_name]['X_test_scaled'] = X_test_scaled
    targets[target_name]['y_train'] = y_train
    targets[target_name]['y_test'] = y_test
    targets[target_name]['scaler'] = scaler

    # Perform HalvingGridSearchCV
    model = MLPRegressor(max_iter=1000, random_state=42)
    halving_search = HalvingGridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1, factor=2, min_resources=50)
    halving_search.fit(X_train_scaled, y_train)
    best_model = halving_search.best_estimator_

    # Make predictions using the best model
    y_pred = best_model.predict(X_test_scaled)

    r2 = r2_score(y_test, y_pred)

    # Print results
    print(f'Best Parameters for {target_name}: {halving_search.best_params_}')
    print(f'R^2 Score for {target_name}: {r2:.3f}')
    
    targets[target_name]['best_model'] = best_model
    targets[target_name]['halving_search'] = halving_search

Best Parameters for IMDb Score: {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (200, 100), 'learning_rate': 'constant', 'solver': 'sgd'}\
R^2 Score for IMDb Score: 0.722\
Best Parameters for Rotten Tomatoes Score: {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (100, 50), 'learning_rate': 'constant', 'solver': 'sgd'}\
R^2 Score for Rotten Tomatoes Score: 0.876\
Best Parameters for Metacritic Score: {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (50,), 'learning_rate': 'constant', 'solver': 'sgd'}\
R^2 Score for Metacritic Score: 0.815

# Visualize Best

In [ ]:
import matplotlib.pyplot as plt

imdb_best = targets[target_im]['best_model']
rt_best = targets[target_rt]['best_model']
mc_best = targets[target_mc]['best_model']

# Create a figure with subplots
plt.figure(figsize=(15, 5))

# Plot IMDb scores using the best model
plt.subplot(131)
y_pred_imdb_best = imdb_best.predict(targets[target_im]['X_test_scaled'])
plt.scatter(targets[target_im]['y_test'], y_pred_imdb_best, alpha=0.6)
plt.plot([targets[target_im]['y_test'].min(), targets[target_im]['y_test'].max()], 
         [targets[target_im]['y_test'].min(), targets[target_im]['y_test'].max()], 'r--', lw=2)
plt.xlabel('Actual IMDb Score')
plt.ylabel('Predicted IMDb Score')
plt.title('Best IMDb Scores')

# Plot Metacritic scores using the best model
plt.subplot(132)
y_pred_mc_best = mc_best.predict(targets[target_mc]['X_test_scaled'])
plt.scatter(targets[target_mc]['y_test'], y_pred_mc_best, alpha=0.6)
plt.plot([targets[target_mc]['y_test'].min(), targets[target_mc]['y_test'].max()], 
         [targets[target_mc]['y_test'].min(), targets[target_mc]['y_test'].max()], 'r--', lw=2)
plt.xlabel('Actual Metacritic Score')
plt.ylabel('Predicted Metacritic Score')
plt.title('Best Metacritic Scores')

# Plot Rotten Tomatoes scores using the best model
plt.subplot(133)
y_pred_rt_best = rt_best.predict(targets[target_rt]['X_test_scaled'])
plt.scatter(targets[target_rt]['y_test'], y_pred_rt_best, alpha=0.6)
plt.plot([targets[target_rt]['y_test'].min(), targets[target_rt]['y_test'].max()], 
         [targets[target_rt]['y_test'].min(), targets[target_rt]['y_test'].max()], 'r--', lw=2)
plt.xlabel('Actual Rotten Tomatoes Score')
plt.ylabel('Predicted Rotten Tomatoes Score')
plt.title('Best Rotten Tomatoes Scores')

plt.tight_layout()
plt.show()